In [1]:
import math
from dataclasses import dataclass
import torch
import torch.nn as nn
from torch.nn import functional as F
from vocab import tokens

In [41]:
import math
import torch
import torch.nn as nn
from vocab import tokens

class GPT(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.max_tokens     = 12
        self.context_length = 16
        self.vocab_size     = 209
        self.n_layers       = 2      # agora duas camadas
        self.n_heads        = 4      # agora quatro cabeças
        self.n_embd         = 16

        # Embeddings
        self.wte = nn.Embedding(self.vocab_size, self.n_embd)
        self.wpe = nn.Embedding(self.context_length, self.n_embd)

        # MLP
        self.fc1 = nn.Linear(self.n_embd, 2 * self.n_embd)
        self.gelu = nn.GELU(approximate='tanh')
        self.fc2 = nn.Linear(2 * self.n_embd, self.n_embd)

        # Normalização
        self.ln = nn.LayerNorm(self.n_embd)
        self.ln.weight.requires_grad = False
        self.ln.bias.requires_grad = False

        # Atenção
        self.qkv_proj = nn.Linear(self.n_embd, 3 * self.n_embd)
        self.out_proj = nn.Linear(self.n_embd, self.n_embd)
        self.head_dim = self.n_embd // self.n_heads

        # Máscara causal
        self.mask = torch.tril(torch.ones(self.context_length,
                                          self.context_length)
                               ).view(self.context_length,
                                      self.context_length)

        # Final head
        self.final_ln = nn.LayerNorm(self.n_embd)
        self.lm_head   = nn.Linear(self.n_embd, self.vocab_size)

        assert self.n_embd % self.n_heads == 0, "n_embd deve ser divisível por n_heads"

    def init_tokens(self, tokens_list):
        self.tokens_list = tokens_list
        self.tokens_vocab = {token: idx for idx, token in enumerate(tokens)}
        self.idx_to_token = {idx: token for token, idx in self.tokens_vocab.items()}

    def mlp(self, x):
        x = self.fc1(x)
        x = self.gelu(x)
        x = self.fc2(x)
        return x

    def layer_norm(self, x):
        return self.ln(x)

    def self_attention(self, x):
        # x: (T, C)
        T, C = x.size()
        qkv = self.qkv_proj(x)               # (T, 3*C)
        q, k, v = qkv.chunk(3, dim=1)        # cada um (T, C)

        # projeção multi-head
        q = q.view(T, self.n_heads, self.head_dim).transpose(0,1)  # (nh, T, hd)
        k = k.view(T, self.n_heads, self.head_dim).transpose(0,1)
        v = v.view(T, self.n_heads, self.head_dim).transpose(0,1)

        # produto escalar e escala
        att = (q @ k.transpose(-2,-1)) / math.sqrt(self.head_dim)   # (nh, T, T)

        # máscara causal
        att = att.masked_fill(self.mask[:T, :T] == 0, float('-inf'))

        # softmax
        att = torch.softmax(att, dim=-1)

        # aplicação sobre v
        y = att @ v  # (nh, T, hd)

        # concatena heads
        y = y.transpose(0,1).contiguous().view(T, C)  # (T, C)

        # projeção final
        return self.out_proj(y)

    def tokens_idx(self, tokens_chosen):
        # Pegar os índices correspondentes
        indices = [self.tokens_vocab[token] for token in tokens_chosen]

        # Converter para tensor, se quiser passar ao modelo
        self.indices_tensor = torch.tensor(indices, dtype=torch.long)

    def forward(self):
        # 1) lookup embeddings
        idx = self.indices_tensor         # (T,)
        T   = idx.size(0)
        tok_emb = self.wte(idx)           # (T, n_embd)
        pos_emb = self.wpe(torch.arange(T, device=idx.device))  # (T, n_embd)
        x = tok_emb + pos_emb             # (T, n_embd)

        # 2) aplica N camadas de (LN → Atenção → Residual → LN → MLP → Residual)
        for _ in range(self.n_layers):
            # pré-atenção
            x_norm = self.layer_norm(x)
            attn_out = self.self_attention(x_norm)
            x = x + attn_out

            # pré-MLP
            x_norm = self.layer_norm(x)
            mlp_out = self.mlp(x_norm)
            x = x + mlp_out

        return x[-1]  # vetor do último token (n_embd,)

    def predict_logits(self, x):
        x = self.final_ln(x)  # (n_embd,)
        return self.lm_head(x)  # (vocab_size,)

    def predict_next_token(self):
        self.tokens_idx(self.tokens_list)
        vec = self.forward()               # (n_embd,)
        logits = self.predict_logits(vec)  # (vocab_size,)
        probs = torch.softmax(logits, dim=-1)
        idx  = torch.multinomial(probs, num_samples=1).item()
        return self.idx_to_token[idx]

    def predict_all_sentence(self):
        # guarda cópia do estado original
        original = list(self.tokens_list)

        for step in range(self.max_tokens):
            nt = self.predict_next_token()
            self.tokens_list.append(nt)
            print(f"Passo {step+1}: {' '.join(self.tokens_list)}")

        # recupera o estado original
        self.tokens_list = original
        return list(original)

In [42]:
tokens_list = ["o", "gato", "pequeno"]
model = GPT()
model.init_tokens(tokens_list)

In [16]:
for name, param in model.named_parameters():
    print(f"Nome: {name}")
    print(f"Shape: {param.shape}")
    print(f"Valores: {param}\n")


Nome: wte.weight
Shape: torch.Size([209, 16])
Valores: Parameter containing:
tensor([[-0.1242, -0.1960,  0.0392,  ...,  0.3843,  2.3576,  0.7143],
        [ 0.9828,  2.3151, -0.1122,  ...,  0.9089, -0.8447, -0.6665],
        [ 0.4429,  0.1570,  1.1231,  ...,  0.7481,  1.2063, -1.7450],
        ...,
        [ 0.1875, -0.1669, -0.4234,  ...,  0.3493, -1.2399, -2.1087],
        [ 0.7212, -0.1140, -0.9392,  ...,  0.1211, -1.4641,  0.4216],
        [ 0.4343,  0.4413, -0.4894,  ...,  0.9323, -0.7992, -0.1888]],
       requires_grad=True)

Nome: wpe.weight
Shape: torch.Size([16, 16])
Valores: Parameter containing:
tensor([[ 1.1967, -0.2463, -0.8689,  0.1212, -2.2817,  1.1339,  0.4178,  1.9397,
          1.5714,  1.5718, -0.7243, -0.1788,  0.3221, -0.1210,  0.8099,  0.3727],
        [-0.6813, -1.4569, -0.5004, -1.0607, -1.5872, -0.2846, -0.5316, -0.2955,
         -0.1098,  1.5127,  1.5372, -1.2987,  0.9330, -0.6001, -1.4782,  0.1846],
        [-1.1192, -0.2405,  0.4051, -0.1834, -0.3536,  0.73

Valores: Parameter containing:
tensor([-0.1917,  0.1750,  0.1953, -0.0810,  0.1231,  0.0821,  0.1502, -0.1653,
         0.1831, -0.0937, -0.2274, -0.1788,  0.0564,  0.1008, -0.0743, -0.0977,
         0.1816, -0.2155,  0.1717, -0.0980, -0.1776, -0.1639, -0.2377,  0.0952,
         0.0115,  0.1213, -0.0215,  0.0009,  0.1063, -0.1983,  0.0089, -0.1568,
        -0.1502,  0.0312, -0.0082, -0.2336, -0.1698,  0.0581,  0.2342, -0.2106,
         0.2353,  0.1917,  0.2035, -0.0433,  0.1964, -0.2007, -0.1430,  0.0583],
       requires_grad=True)

Nome: out_proj.weight
Shape: torch.Size([16, 16])
Valores: Parameter containing:
tensor([[ 0.2374,  0.2366,  0.2150,  0.1221,  0.2267, -0.1625,  0.2420,  0.0584,
         -0.2139, -0.0107,  0.0479,  0.0275, -0.2230, -0.1187,  0.0731, -0.1701],
        [ 0.1832,  0.0262,  0.0428, -0.1969, -0.1900,  0.2169,  0.1331,  0.2452,
          0.2212,  0.2092,  0.1304, -0.1415, -0.0077, -0.1871,  0.2109,  0.0123],
        [-0.1489, -0.1997,  0.2301,  0.0735,  0.1892,

In [17]:
print(model)

GPT(
  (wte): Embedding(209, 16)
  (wpe): Embedding(16, 16)
  (fc1): Linear(in_features=16, out_features=32, bias=True)
  (gelu): GELU(approximate='tanh')
  (fc2): Linear(in_features=32, out_features=16, bias=True)
  (ln): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (qkv_proj): Linear(in_features=16, out_features=48, bias=True)
  (out_proj): Linear(in_features=16, out_features=16, bias=True)
  (final_ln): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
  (lm_head): Linear(in_features=16, out_features=209, bias=True)
)


In [44]:
model.predict_next_token()

'abrir'

In [30]:
model.idx_to_token

{0: 'eu',
 1: 'você',
 2: 'ele',
 3: 'ela',
 4: 'nós',
 5: 'vocês',
 6: 'eles',
 7: 'elas',
 8: 'me',
 9: 'te',
 10: 'se',
 11: 'nos',
 12: 'vos',
 13: 'lhe',
 14: 'lhes',
 15: 'mim',
 16: 'ti',
 17: 'si',
 18: 'ser',
 19: 'estar',
 20: 'ter',
 21: 'haver',
 22: 'fazer',
 23: 'poder',
 24: 'dizer',
 25: 'ver',
 26: 'dar',
 27: 'saber',
 28: 'querer',
 29: 'chegar',
 30: 'passar',
 31: 'dever',
 32: 'ficar',
 33: 'deixar',
 34: 'pensar',
 35: 'vir',
 36: 'conhecer',
 37: 'casa',
 38: 'tempo',
 39: 'dia',
 40: 'mundo',
 41: 'homem',
 42: 'mulher',
 43: 'vida',
 44: 'mão',
 45: 'olho',
 46: 'palavra',
 47: 'caminho',
 48: 'gato',
 49: 'cachorro',
 50: 'carro',
 51: 'livro',
 52: 'porta',
 53: 'rua',
 154: 'trabalho',
 55: 'dinheiro',
 56: 'noite',
 57: 'bom',
 58: 'mau',
 59: 'feliz',
 60: 'triste',
 61: 'grande',
 62: 'pequeno',
 63: 'novo',
 64: 'velho',
 65: 'forte',
 66: 'fraco',
 67: 'belo',
 68: 'feio',
 69: 'inteligente',
 70: 'rápido',
 71: 'lento',
 72: 'em',
 73: 'de',
 74: 'com

In [47]:
model.predict_all_sentence()

Passo 1: o gato pequeno vamos
Passo 2: o gato pequeno vamos ali
Passo 3: o gato pequeno vamos ali um
Passo 4: o gato pequeno vamos ali um após
Passo 5: o gato pequeno vamos ali um após escola
Passo 6: o gato pequeno vamos ali um após escola experiência
Passo 7: o gato pequeno vamos ali um após escola experiência conhecer
Passo 8: o gato pequeno vamos ali um após escola experiência conhecer escola
Passo 9: o gato pequeno vamos ali um após escola experiência conhecer escola amar
Passo 10: o gato pequeno vamos ali um após escola experiência conhecer escola amar a
Passo 11: o gato pequeno vamos ali um após escola experiência conhecer escola amar a felizes
Passo 12: o gato pequeno vamos ali um após escola experiência conhecer escola amar a felizes os


['o', 'gato', 'pequeno']